In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from utils.nn.optimizer import Adam
from utils.nn.neuron import Activation
from utils.nn.layer import Dense, Dropout
from utils.nn.sequential import Sequential
from utils.nn.losses import categorical_crossentropy_loss
from utils.nn.initializer import Initializer, InitializationType

In [ ]:
mnist_dataset = load_digits()
pd.DataFrame({'features': list(mnist_dataset.data), 'label': mnist_dataset.target})

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 3))

for i, (image, image_target) in enumerate(zip(mnist_dataset.data[:5], mnist_dataset.target[:5])):
    image_2d = image.reshape(8, 8)
    axes[i].imshow(image_2d, cmap='gray')
    axes[i].set_title(f"Digit: {image_target}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
x = mnist_dataset.data
y = mnist_dataset.target.reshape(-1, 1)

onehot_encoder = OneHotEncoder()
y_onehot = onehot_encoder.fit_transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(x, y_onehot, test_size=0.2)

print(f"X_train: {X_train.shape} y_train: {y_train.shape}")
print(f"X_test: {X_test.shape} y_test: {y_test.shape}")

In [ ]:
model = Sequential(
    layers=[
        
        Dense(
            shape=(64, 128),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=64, fan_out=128)
        ),

        Dropout(rate=0.3, input_size=128),
        
        Dense(
            shape=(128, 64),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=128, fan_out=64)
        ),

        Dropout(rate=0.4, input_size=64),
        
        Dense(
            shape=(64, 32),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=64, fan_out=32)
        ),
        
        Dropout(rate=0.5, input_size=32),

        Dense(
            shape=(32, 10),
            activation=Activation.SOFTMAX,
            initializer=Initializer(fill_type=InitializationType.GLOROT_NORMAL, fan_in=32, fan_out=10)
        )
    
    ]
)

model.summary()

In [ ]:
optimizer = Adam(params=model.parameters(), learning_rate=1e-3)

model.fit(
    x_train=X_train,
    y_train=y_train,
    optimizer=optimizer,
    loss_func=categorical_crossentropy_loss,
    epochs=100,
    batch_size=32,
    metric="accuracy",
    x_validate=X_test,
    y_validate=y_test,
    display_interval=10
)

In [ ]:
y_test_predictions = model.forward_batch(X_test)
y_test_prediction_labels = [np.argmax(pred) for pred in y_test_predictions]
y_test_true_labels = [np.argmax(yt) for yt in y_test]

In [ ]:
def visualize_prediction(x_test, y_true, y_pred, num_samples=10, grid_cols=5):

    """
    Visualize MNIST test samples with true and predicted labels
    
    Parameters:
    - x_test: Test images (numpy array)
    - y_true: True labels
    - y_pred: Predicted labels
    - num_samples: Number of samples to display (default: 10)
    - grid_cols: Number of columns in the grid (default: 5)
    """
    
    # Calculate grid dimensions
    grid_rows = (num_samples + grid_cols - 1) // grid_cols
    
    # Create subplots
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(15, 3 * grid_rows))
    
    # Handle case where we have only one row
    if grid_rows == 1:
        axes = axes.reshape(1, -1)
    
    # Flatten axes for easier indexing if we have multiple rows
    axes_flat = axes.flatten()
    
    for i in range(num_samples):
        # Reshape image to 2D (assuming 8x8 for sklearn digits, adjust if needed)
        if len(x_test[i].shape) == 1:
            # If flattened, reshape to square (common sizes: 8x8=64, 28x28=784)
            img_size = int(np.sqrt(x_test[i].shape[0]))
            image_2d = x_test[i].reshape(img_size, img_size)
        else:
            image_2d = x_test[i]
        
        # Display image
        axes_flat[i].imshow(image_2d, cmap='gray')
        
        # Set title with true and predicted labels
        # Color code: green if correct, red if incorrect
        color = 'green' if y_true[i] == y_pred[i] else 'red'
        axes_flat[i].set_title(f'True: {y_true[i]}, Pred: {y_pred[i]}', 
                              color=color, fontsize=10, fontweight='bold')
        axes_flat[i].axis('off')
    
    # Hide any unused subplots
    for i in range(num_samples, len(axes_flat)):
        axes_flat[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_prediction(X_test, y_true=y_test_true_labels, y_pred=y_test_prediction_labels)